# Data Visualization Module
The purpose of this notebook is to act as a module that can be imported into the Master Notebook to visualize various data.

## Fine Particulate Matter (PM2.5) Time Series
The following function plots PM2.5 concentration as either $\mu$g/m$^3$ or AQI. The background is formatted such that the breakpoints for AQI are included. Breakpoints for AQI can be found [here](http://aqicn.org/faq/2013-09-09/revised-pm25-aqi-breakpoints/). 

In [2]:
def getFinePMTimeSeries(x, y, ylabel, day_no=0, figure_size=(16,8)):
    '''
    Inputs:
        - x: numpy DateTime array holding the independent variable data
        - y: numpy float array holding the PM2.5 concentration or AQI
        - ylabel: string representing the y-axis label 
        - day_no: integer for what day in the series it is
        - figure_size: a tuple holding the figure size; default is (16,8)
    Returns a time series plot of PM2.5 Concentration or AQI with AQI breakpoints included as background
    '''
    # Plotting
    fig, ax = plt.subplots(figsize = figure_size)
    ax.plot(x,y,color='black',linewidth=3,label='PM2.5')

    ## Formatting x-axis
    ax.set_xlim([x[0],x[-1]])
    ### Formatting datetime labels
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d %H:%M'))
    ax.xaxis.set_major_locator(mdates.DayLocator())
    ax.xaxis.set_minor_formatter(mdates.DateFormatter('%H:%M'))
    ax.xaxis.set_minor_locator(mdates.HourLocator(byhour=range(2,24,2)))
    ax.set_xlabel('Time')
    ax.xaxis.set_tick_params(rotation=-30)
    
    ## Formatting y-axis
    ax.set_ylim([0,500])
    ax.set_ylabel(ylabel)
    
    ## Formatting remaining aspects of figure
    ax.set_title('Day ' + str(day_no) + ': ' + str(x[2].date()))
    ### AQI breakpoints
    bp_c_start = [0,12,35.5,55.5,150.5,250.5]
    bp_c_end = [12,35.4,55.4,150.4,250.4,500]
    bp_aqi_start = [0,51,101,151,201,301]
    bp_aqi_end = [50,100,150,200,300,500]
    if ylabel == 'Air Quality Index':
        bp_start = bp_aqi_start
        bp_end = bp_aqi_end
    else:
        bp_start = bp_c_start
        bp_end = bp_c_end
    ax.axhspan(bp_start[0],bp_end[0],color='green',alpha=0.3,label='Good')
    ax.axhspan(bp_start[1],bp_end[1],color='yellow',alpha=0.3,label='Moderate')
    ax.axhspan(bp_start[2],bp_end[2],color='orange',alpha=0.3,label='Unhealthy for Children/Elderly')
    ax.axhspan(bp_start[3],bp_end[3],color='red',alpha=0.3,label='Unhealthy')
    ax.axhspan(bp_start[4],bp_end[4],color='purple',alpha=0.3,label='Very Unhealthy')
    ax.axhspan(bp_start[5],bp_end[5],color='maroon',alpha=0.3,label='Hazardous')
    ax.legend()

## Sleep Stages Bar Chart
The following function plots the relative percentage of wake, rem, and non-rem stages. The background corresponds to the approximate ranges that qualify as proper sleep.

In [68]:
def getSleepStagePercentages(sleep_metrics):
    '''
    Inputs:
        - sleep_metrics: a dataframe that holds different sleep metrics, indexed by the night
    Returns a bar chart showing the relative percentage of awake, non-rem, and rem sleep stages
    '''
    # Plotting
    fig, ax = plt.subplots(figsize=(16,8))
    index = np.arange(len(sleep_metrics))
    p1 = ax.bar(index,sleep_metrics['Awake %'],bottom=sleep_metrics['Non-REM %']+sleep_metrics['REM %'], color='pink',edgecolor='k',label='Awake')
    p2 = ax.bar(index,sleep_metrics['REM %'], bottom=sleep_metrics['Non-REM %'],color='c',edgecolor='k',label='REM')
    p3 = ax.bar(index,sleep_metrics['Non-REM %'], color='navy',edgecolor='k',label='Non-REM')

    ##
    rect_no = 0
    for rect in p1:
        x_pos = rect.get_x() + rect.get_width()/2.0
        y_pos = 5
        height = rect.get_height()
        text = 'Sleep Efficiency: ' + str(round(sleep_metrics['Efficiency'][rect_no],1)) + '%'
        plt.text(x_pos, y_pos, text, ha='center', va='bottom',color='white')
        rect_no += 1
    
    ## Formatting x-axis
    ax.xaxis_date()
    ax.set_xlabel('Date')
    date_ticks = []
    for night in sleep_metrics.index:
        date_ticks.append(str(night.month) + '/' + str(night.day))
    plt.xticks(index,date_ticks)

    ## Formatting y-axis
    ax.set_yticks(np.arange(0,105,10))
    ax.set_ylabel('Percent of Sleep Spent in Each Stage')
        
    ax.legend()

In [63]:
def getSleepStageBenchmark(sleep_metrics):
    '''
    Inputs:
        - sleep_metrics: a dataframe that holds different sleep metrics, indexed by the night
    Returns a bar chart that shows the percentage spent in each sleep stage compared against the recommended levels
    '''
    # Plotting
    fig, ax = plt.subplots(figsize=(16,8))
    ## Generating bar chart details
    index = np.arange(len(sleep_metrics))
    bar_width = 0.2
    pad = 0.1
    
    ## Creating the bars and the shaded areas
    rect_shade1 = ax.bar(index-pad-bar_width,15,bottom=5,width=bar_width+pad/2,color='pink',alpha=0.3,hatch='/',edgecolor='k',label='Typical Awake %')
    p1 = ax.bar(index-pad-bar_width,sleep_metrics['Awake %'],width=bar_width,color='pink',edgecolor='k',label='Your Awake %')
    rect_shade2 = ax.bar(index,10,bottom=13,width=bar_width+pad/2,color='c',alpha=0.3,hatch='/',edgecolor='k',label='Recommended REM %')
    p2 = ax.bar(index,sleep_metrics['REM %'],width=bar_width,color='c',edgecolor='k',label='Your REM %')
    rect_shad3 = ax.bar(index+pad+bar_width,83-52,bottom=52,width=bar_width+pad/2,color='navy',alpha=0.3,hatch='/',edgecolor='k',label='Recommended Non-REM %')
    p3 = ax.bar(index+pad+bar_width,sleep_metrics['Non-REM %'],width=bar_width,color='navy',edgecolor='k',label='Your Non-REM %')

    ## Formatting x-axis
    ax.xaxis_date()
    ax.set_xlabel('Date')
    date_ticks = []
    for night in sleep_metrics.index:
        date_ticks.append(str(night.month) + '/' + str(night.day))
    plt.xticks(index,date_ticks)
    
    ## Formatting y-axis
    ax.set_ylabel('Percent of Time Spent in Sleep Stage')
    ax.set_ylim([0,100])
    ax.set_yticks(np.arange(0,105,10))
    ax.legend()